<a href="https://colab.research.google.com/github/viincci/Jstest/blob/main/RuleTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q yfinance

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import logging
import yfinance as yf

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('smc_strategy')

In [10]:

class SMCBacktester:
    def __init__(self, df, initial_balance=10000, risk_per_trade=0.02):
        """
        Initialize SMC Backtester with historical data and configuration

        Parameters:
        df (pandas.DataFrame): OHLC data with columns 'open', 'high', 'low', 'close'
        initial_balance (float): Starting account balance
        risk_per_trade (float): Risk percentage per trade (0.01 = 1%)
        """
        logger.info(f"Initializing SMC Strategy Backtester with {len(df)} candles")
        print(f"Initializing SMC Strategy Backtester with {len(df)} candles")

       # Ensure dataframe has required columns
        required_cols = ['Open', 'High', 'Low', 'Close']

        # Check if it's a multi-index and convert
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        for col in required_cols:
            if col not in df.columns:
                raise ValueError(f"DataFrame missing required column: {col}")
        # Initialize data and account parameters
        self.df = df.copy()
        # Convert column names to lowercase for consistency with original code
        self.df.columns = [col.lower() for col in self.df.columns]
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.risk_per_trade = risk_per_trade

        # Add columns for SMC analysis
        self.df['higher_high'] = False
        self.df['lower_low'] = False
        self.df['bos_up'] = False
        self.df['bos_down'] = False
        self.df['choch_up'] = False
        self.df['choch_down'] = False
        self.df['bullish_fvg'] = False
        self.df['bearish_fvg'] = False

        # Trading variables
        self.in_trade = False
        self.entry_price = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.position_size = 0
        self.trade_type = None  # 'long' or 'short'

        # Performance tracking
        self.trades = []
        self.equity_curve = [initial_balance]

        logger.info(f"Strategy initialized with {initial_balance} account balance and {risk_per_trade*100}% risk per trade")
        print(f"Strategy initialized with {initial_balance} account balance and {risk_per_trade*100}% risk per trade")

    def identify_structure(self):
        """Identify market structure including highs, lows, BOS and CHoCH"""
        logger.info("Identifying market structure")
        print("Identifying market structure")
        df = self.df

        # Identify Higher Highs and Lower Lows (using a 5-candle lookback)
        window = 5
        for i in range(window, len(df)):
            # Higher High
            if df.iloc[i]['high'] > max(df.iloc[i-window:i]['high']):
                df.loc[df.index[i], 'higher_high'] = True

            # Lower Low
            if df.iloc[i]['low'] < min(df.iloc[i-window:i]['low']):
                df.loc[df.index[i], 'lower_low'] = True

        # Identify Break of Structure (BOS)
        prev_structure_high = df.iloc[0]['high']
        prev_structure_low = df.iloc[0]['low']
        structure_points_high = []
        structure_points_low = []

        for i in range(1, len(df)):
            current_high = df.iloc[i]['high']
            current_low = df.iloc[i]['low']

            # Track significant structure points
            if df.iloc[i]['higher_high']:
                structure_points_high.append((i, current_high))

            if df.iloc[i]['lower_low']:
                structure_points_low.append((i, current_low))

            # BOS Up: Price breaks above recent structure high
            if len(structure_points_high) >= 2:
                last_high_idx, last_high = structure_points_high[-1]
                prev_high_idx, prev_high = structure_points_high[-2]

                if current_low > prev_high and i > last_high_idx + 1:
                    df.loc[df.index[i], 'bos_up'] = True
                    logger.info(f"Bullish BOS detected at index {i}, price: {current_low}")
                    print(f"Bullish BOS detected at index {i}, price: {current_low}")

            # BOS Down: Price breaks below recent structure low
            if len(structure_points_low) >= 2:
                last_low_idx, last_low = structure_points_low[-1]
                prev_low_idx, prev_low = structure_points_low[-2]

                if current_high < prev_low and i > last_low_idx + 1:
                    df.loc[df.index[i], 'bos_down'] = True
                    logger.info(f"Bearish BOS detected at index {i}, price: {current_high}")
                    print(f"Bearish BOS detected at index {i}, price: {current_high}")

        # Identify Change of Character (CHoCH)
        for i in range(window+1, len(df)):
            # Bullish CHoCH: After BOS up, creates higher low
            if df.iloc[i-1]['bos_up']:
                recent_lows = df.iloc[i-window:i]['low'].tolist()
                if min(recent_lows[:-1]) < recent_lows[-1]:
                    df.loc[df.index[i], 'choch_up'] = True
                    logger.info(f"Bullish CHoCH detected at index {i}")
                    print(f"Bullish CHoCH detected at index {i}")

            # Bearish CHoCH: After BOS down, creates lower high
            if df.iloc[i-1]['bos_down']:
                recent_highs = df.iloc[i-window:i]['high'].tolist()
                if max(recent_highs[:-1]) > recent_highs[-1]:
                    df.loc[df.index[i], 'choch_down'] = True
                    logger.info(f"Bearish CHoCH detected at index {i}")
                    print(f"Bearish CHoCH detected at index {i}")

        return df

    def identify_fvg(self):
        """Identify Fair Value Gaps (FVGs)"""
        logger.info("Identifying Fair Value Gaps")
        print("Identifying Fair Value Gaps")
        df = self.df

        for i in range(2, len(df)):
            # Bullish FVG: Previous candle's low > Current candle's high
            if df.iloc[i-2]['low'] > df.iloc[i]['high']:
                # Check Gann Box (0-0.5 range for bullish FVGs)
                high_point = df.iloc[i-2]['high']
                low_point = df.iloc[i]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i]['high']
                fvg_high = df.iloc[i-2]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0-0.5 Gann range
                    if 0 <= relative_pos <= 0.5:
                        df.loc[df.index[i], 'bullish_fvg'] = True
                        # Store FVG values as tuple (low, high, candle_index_for_sl)
                        #df.loc[df.index[i], 'bullish_fvg_values'] = (fvg_low, fvg_high, i)
                        # Inside the identify_fvg function:
                        df.loc[df.index[i], 'bullish_fvg_values'] = (fvg_low, fvg_high, I)

                        logger.info(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

            # Bearish FVG: Previous candle's high < Current candle's low
            if df.iloc[i-2]['high'] < df.iloc[i]['low']:
                # Check Gann Box (0.5-1 range for bearish FVGs)
                high_point = df.iloc[i]['high']
                low_point = df.iloc[i-2]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i-2]['high']
                fvg_high = df.iloc[i]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0.5-1 Gann range
                    if 0.5 <= relative_pos <= 1:
                        df.loc[df.index[i], 'bearish_fvg'] = True
                        # Store FVG values as tuple (low, high, candle_index_for_sl)
                        #df.loc[df.index[i], 'bearish_fvg_values'] = (fvg_low, fvg_high, i)
                        df.at[df.index[i], 'bearish_fvg_values'] = (fvg_low, fvg_high, i)
                        logger.info(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

        return df

    def check_fvg_mitigation(self, current_idx):
        """Check if any previously identified FVGs have been mitigated"""
        df = self.df

        # Loop through all previous candles
        for i in range(current_idx):
            # Check if the candle had a bullish FVG
            if 'bullish_fvg_values' in df.columns and pd.notna(df.iloc[i].get('bullish_fvg_values')):
                fvg_low, fvg_high, sl_idx = df.iloc[i]['bullish_fvg_values']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bullish_fvg_mitigated'] = True
                        logger.info(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        break

            # Check if the candle had a bearish FVG
            if 'bearish_fvg_values' in df.columns and pd.notna(df.iloc[i].get('bearish_fvg_values')):
                fvg_low, fvg_high, sl_idx = df.iloc[i]['bearish_fvg_values']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bearish_fvg_mitigated'] = True
                        logger.info(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        break

        return df

    def execute_trades(self):
        """Execute trades based on SMC signals"""
        logger.info("Starting trade execution backtesting")
        print("Starting trade execution backtesting")
        df = self.df

        # Iterate through each candle for backtesting
        for i in range(5, len(df)):
            current_price = df.iloc[i]['close']

            # Check if we're in a trade
            if self.in_trade:
                # Check if stop loss hit
                if (self.trade_type == 'long' and df.iloc[i]['low'] <= self.stop_loss) or \
                   (self.trade_type == 'short' and df.iloc[i]['high'] >= self.stop_loss):
                    # Stop loss hit
                    self.exit_trade(i, self.stop_loss, 'stop_loss')

                # Check if take profit hit
                elif (self.trade_type == 'long' and df.iloc[i]['high'] >= self.take_profit) or \
                     (self.trade_type == 'short' and df.iloc[i]['low'] <= self.take_profit):
                    # Take profit hit
                    self.exit_trade(i, self.take_profit, 'take_profit')

            else:
                # Update FVG mitigation status
                self.check_fvg_mitigation(i)

                # Check for new trade setups

                # Bullish setup: BOS up + CHoCH up + unmitigated bullish FVG
                if df.iloc[i-1]['bos_up'] and df.iloc[i]['choch_up']:
                    # Look back for unmitigated bullish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bullish_fvg', False) and not df.iloc[j].get('bullish_fvg_mitigated', False):
                            fvg_low, fvg_high, sl_idx = df.iloc[j]['bullish_fvg_values']

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the low of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['low']

                                # Find recent structure low for take profit
                                recent_lows = df.iloc[i-20:i]['low'].tolist()
                                min_idx = recent_lows.index(min(recent_lows))
                                take_profit = df.iloc[i-20+min_idx]['low']

                                # Enter long trade
                                self.enter_trade(i, current_price, stop_loss, take_profit, 'long')
                                break

                # Bearish setup: BOS down + CHoCH down + unmitigated bearish FVG
                if df.iloc[i-1]['bos_down'] and df.iloc[i]['choch_down']:
                    # Look back for unmitigated bearish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bearish_fvg', False) and not df.iloc[j].get('bearish_fvg_mitigated', False):
                            fvg_low, fvg_high, sl_idx = df.iloc[j]['bearish_fvg_values']

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the high of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['high']

                                # Find recent structure high for take profit
                                recent_highs = df.iloc[i-20:i]['high'].tolist()
                                max_idx = recent_highs.index(max(recent_highs))
                                take_profit = df.iloc[i-20+max_idx]['high']

                                # Enter short trade
                                self.enter_trade(i, current_price, stop_loss, take_profit, 'short')
                                break

        # Close any open trade at the end of testing
        if self.in_trade:
            self.exit_trade(len(df) - 1, df.iloc[-1]['close'], 'end_of_test')

        logger.info(f"Trade execution completed with {len(self.trades)} trades")
        print(f"Trade execution completed with {len(self.trades)} trades")

        return self.trades, self.equity_curve

    def enter_trade(self, index, price, stop_loss, take_profit, trade_type):
        """Enter a new trade with defined parameters"""
        # Calculate position size based on risk
        risk_amount = self.balance * self.risk_per_trade
        trade_risk = abs(price - stop_loss)
        self.position_size = risk_amount / trade_risk

        # Record trade details
        self.in_trade = True
        self.entry_price = price
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.trade_type = trade_type
        self.entry_index = index

        trade_info = {
            'entry_date': self.df.index[index],
            'entry_index': index,
            'entry_price': price,
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'position_size': self.position_size,
            'risk_amount': risk_amount,
            'type': trade_type,
            'balance_before': self.balance
        }

        logger.info(f"Entered {trade_type} trade at price {price}, SL: {stop_loss}, TP: {take_profit}, Risk: {risk_amount:.2f}")
        print(f"Entered {trade_type} trade at price {price}, SL: {stop_loss}, TP: {take_profit}, Risk: {risk_amount:.2f}")

        return trade_info

    def exit_trade(self, index, price, exit_reason):
        """Exit current trade and update performance metrics"""
        # Calculate profit/loss
        if self.trade_type == 'long':
            pnl = (price - self.entry_price) * self.position_size
        else:  # short
            pnl = (self.entry_price - price) * self.position_size

        # Update balance
        self.balance += pnl
        self.equity_curve.append(self.balance)

        # Record trade outcome
        trade = {
            'entry_date': self.df.index[self.entry_index],
            'entry_index': self.entry_index,
            'entry_price': self.entry_price,
            'exit_date': self.df.index[index],
            'exit_index': index,
            'exit_price': price,
            'stop_loss': self.stop_loss,
            'take_profit': self.take_profit,
            'position_size': self.position_size,
            'pnl': pnl,
            'exit_reason': exit_reason,
            'type': self.trade_type,
            'balance_after': self.balance
        }

        self.trades.append(trade)

        # Reset trade variables
        self.in_trade = False
        self.entry_price = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.position_size = 0
        self.trade_type = None

        logger.info(f"Exited trade at price {price}, Reason: {exit_reason}, P/L: {pnl:.2f}, New Balance: {self.balance:.2f}")
        print(f"Exited trade at price {price}, Reason: {exit_reason}, P/L: {pnl:.2f}, New Balance: {self.balance:.2f}")

        return trade

    def calculate_performance(self):
        """Calculate and return performance metrics"""
        if not self.trades:
            logger.warning("No trades to calculate performance metrics")
            print("No trades to calculate performance metrics")
            return {
                'total_trades': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'total_return_pct': 0,
                'max_drawdown_pct': 0
            }

        # Calculate win rate
        winning_trades = [t for t in self.trades if t['pnl'] > 0]
        win_rate = len(winning_trades) / len(self.trades)

        # Calculate profit factor
        gross_profit = sum([t['pnl'] for t in self.trades if t['pnl'] > 0])
        gross_loss = abs(sum([t['pnl'] for t in self.trades if t['pnl'] <= 0]))
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

        # Calculate total return
        total_return = (self.balance - self.initial_balance) / self.initial_balance

        # Calculate maximum drawdown
        peak = self.initial_balance
        max_drawdown = 0

        for balance in self.equity_curve:
            if balance > peak:
                peak = balance
            drawdown = (peak - balance) / peak
            max_drawdown = max(max_drawdown, drawdown)

        performance = {
            'total_trades': len(self.trades),
            'winning_trades': len(winning_trades),
            'losing_trades': len(self.trades) - len(winning_trades),
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'total_return': self.balance - self.initial_balance,
            'total_return_pct': total_return * 100,
            'max_drawdown_pct': max_drawdown * 100,
            'final_balance': self.balance
        }

        logger.info(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
                     f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")
        print(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
              f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")

        return performance

    def visualize_results(self, start_idx=0, end_idx=None):
        """Visualize backtesting results with trades and SMC patterns"""
        if end_idx is None:
            end_idx = len(self.df)

        # Create figure with subplots
        fig, ax = plt.subplots(figsize=(15, 8))

        # Plot price data
        subset = self.df.iloc[start_idx:end_idx]
        ax.plot(subset.index, subset['close'], label='Close Price', color='black', linewidth=1)

        # Plot FVGs
        for i in range(start_idx, min(end_idx, len(self.df))):
            if 'bullish_fvg_values' in self.df.columns and pd.notna(self.df.iloc[i].get('bullish_fvg_values')):
                fvg_low, fvg_high, _ = self.df.iloc[i]['bullish_fvg_values']
                mitigated = self.df.iloc[i].get('bullish_fvg_mitigated', False)
                color = 'lightgreen' if not mitigated else 'darkgreen'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

            if 'bearish_fvg_values' in self.df.columns and pd.notna(self.df.iloc[i].get('bearish_fvg_values')):
                fvg_low, fvg_high, _ = self.df.iloc[i]['bearish_fvg_values']
                mitigated = self.df.iloc[i].get('bearish_fvg_mitigated', False)
                color = 'lightcoral' if not mitigated else 'darkred'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

        # Plot BOS and CHoCH
        bos_up_idx = subset[subset['bos_up'] == True].index
        bos_down_idx = subset[subset['bos_down'] == True].index
        choch_up_idx = subset[subset['choch_up'] == True].index
        choch_down_idx = subset[subset['choch_down'] == True].index

        ax.scatter(bos_up_idx, subset.loc[bos_up_idx, 'low'], color='green', marker='^', s=100, label='BOS Up')
        ax.scatter(bos_down_idx, subset.loc[bos_down_idx, 'high'], color='red', marker='v', s=100, label='BOS Down')
        ax.scatter(choch_up_idx, subset.loc[choch_up_idx, 'low'], color='blue', marker='^', s=80, label='CHoCH Up')
        ax.scatter(choch_down_idx, subset.loc[choch_down_idx, 'high'], color='purple', marker='v', s=80, label='CHoCH Down')

        # Plot trades
        for trade in self.trades:
            if start_idx <= trade['entry_index'] < end_idx:
                # Entry point
                color = 'green' if trade['type'] == 'long' else 'red'
                marker = '^' if trade['type'] == 'long' else 'v'
                ax.scatter(trade['entry_index'], trade['entry_price'], color=color, marker=marker, s=120, zorder=5)

                # Exit point
                if trade['exit_index'] < end_idx:
                    color = 'green' if trade['pnl'] > 0 else 'red'
                    ax.scatter(trade['exit_index'], trade['exit_price'], color=color, marker='o', s=120, zorder=5)

                    # Connect entry and exit
                    ax.plot([trade['entry_index'], trade['exit_index']],
                           [trade['entry_price'], trade['exit_price']],
                           color=color, linewidth=1, linestyle='--')

                    # Annotate PnL
                    ax.annotate(f"{trade['pnl']:.2f}",
                              (trade['exit_index'], trade['exit_price']),
                              textcoords="offset points",
                              xytext=(0,10),
                              ha='center')

        ax.set_title('SMC Backtest Results')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

        # Plot equity curve
        fig2, ax2 = plt.subplots(figsize=(15, 5))
        ax2.plot(self.equity_curve, label='Account Balance', color='blue')
        ax2.set_title('Equity Curve')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

        plt.tight_layout()
        return fig, fig2

    def run_backtest(self):
        """Run the full backtest process"""
        logger.info("Starting SMC backtest")
        print("Starting SMC backtest")

        # Identify market structure
        self.identify_structure()

        # Identify Fair Value Gaps
        self.identify_fvg()

        # Execute trades based on signals
        self.execute_trades()

        # Calculate performance metrics
        performance = self.calculate_performance()

        logger.info("Backtest completed successfully")
        print("Backtest completed successfully")

        return {
            'trades': self.trades,
            'performance': performance,
            'equity_curve': self.equity_curve,
            'processed_data': self.df
        }

In [8]:
# Use yfinance to get real market data
def run_smc_backtest(ticker="SPY", start_date="2023-01-01", end_date="2023-12-31", initial_balance=10000, risk_per_trade=0.02):
    """
    Run SMC backtesting on real market data from yfinance

    Parameters:
    ticker (str): Stock ticker symbol
    start_date (str): Start date for historical data in YYYY-MM-DD format
    end_date (str): End date for historical data in YYYY-MM-DD format
    initial_balance (float): Starting account balance
    risk_per_trade (float): Risk percentage per trade (0.02 = 2%)
    """
    print(f"Fetching {ticker} market data from {start_date} to {end_date}")

    # Get historical data from yfinance
    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        print(f"No data found for {ticker} in the specified date range")
        return None

    print(f"Downloaded {len(data)} days of market data for {ticker}")

    # Initialize and run backtester
    smc = SMCBacktester(data, initial_balance=initial_balance, risk_per_trade=risk_per_trade)
    results = smc.run_backtest()

    # Print performance summary
    print("\n--- PERFORMANCE SUMMARY ---")
    print(f"Total Trades: {results['performance']['total_trades']}")
    print(f"Winning Trades: {results['performance']['winning_trades']}")
    print(f"Win Rate: {results['performance']['win_rate']:.2%}")
    print(f"Profit Factor: {results['performance']['profit_factor']:.2f}")
    print(f"Total Return: ${results['performance']['total_return']:.2f} ({results['performance']['total_return_pct']:.2f}%)")
    print(f"Max Drawdown: {results['performance']['max_drawdown_pct']:.2f}%")
    print(f"Final Balance: ${results['performance']['final_balance']:.2f}")

    # Visualize results
    smc.visualize_results()
    plt.show()

    return results, smc

In [11]:
results, smc = run_smc_backtest()

[*********************100%***********************]  1 of 1 completed

Fetching SPY market data from 2023-01-01 to 2023-12-31
Downloaded 250 days of market data for SPY
Initializing SMC Strategy Backtester with 250 candles
Strategy initialized with 10000 account balance and 2.0% risk per trade
Starting SMC backtest
Identifying market structure
Bullish BOS detected at index 125, price: 431.71554283704035
Bullish BOS detected at index 127, price: 428.1984802602594
Bullish BOS detected at index 129, price: 429.3121862073736


Bullish CHoCH detected at index 126
Bullish CHoCH detected at index 128
Bullish CHoCH detected at index 130
Identifying Fair Value Gaps


ValueError: Must have equal len keys and value when setting with an iterable